In [273]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize

In [274]:
class FashionMNISTtask1(datasets.FashionMNIST):
  def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
    super(FashionMNISTtask1, self).__init__(root, train=train, transform=transform,
                                            target_transform=target_transform,download=download)
    self.classes = self.classes[:6]

  def __getitem__(self, index):
    img, target = super(FashionMNISTtask1, self).__getitem__(index)
    if target < 6:
        return img, target
    else:
        return img, -1

In [275]:
class FashionMNISTtask2(datasets.FashionMNIST):
    def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        super(FashionMNISTtask2, self).__init__(root, train=train, transform=transform,
                                                target_transform=target_transform,download=download)
        self.classes = self.classes[6:]

    def __getitem__(self, index):
        img, target = super(FashionMNISTtask2, self).__getitem__(index)
        if target >= 6:
            return img, target
        else:
            return img, -1

In [276]:
transform = torchvision.transforms.Compose([ToTensor(), Normalize((0.5), (0.5))])

train_dataset_1 = FashionMNISTtask1(root='./data1', train=True, transform=transform, download=True)
test_dataset_1 = FashionMNISTtask1(root='./data1', train=False, transform=transform, download=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Function to display an image
def show_image(image):
    # Convert the PyTorch tensor to a numpy array and reshape it to (28, 28)
    image = np.squeeze(image.numpy())
    # Plot the image
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.show()

# Example: Display the first 10 images from the training dataset
for i in range(10):
    image, target = train_dataset_1[i]
    print("Image shape:", image.shape, "Label:", target)
    show_image(image)

In [278]:
train_dataset_2 = FashionMNISTtask2(root='./data2', train=True, transform=transform, download=True)
test_dataset_2 = FashionMNISTtask2(root='./data2', train=False, transform=transform, download=True)
test_dataset_3 = datasets.FashionMNIST(
    root='data3',
    train=False,
    download=True,
    transform=transform
)

In [279]:
train_dataset_filtered_first = [data for data in train_dataset_1 if data[1] != -1]
test_dataset_filtered_first = [data for data in test_dataset_1 if data[1] != -1]

train_dataset_filtered_second = [data for data in train_dataset_2 if data[1] != -1]
test_dataset_filtered_second = [data for data in test_dataset_2 if data[1] != -1]

In [280]:
unique_labels_1 = set()
unique_labels_2 = set()

for _, target in train_dataset_filtered_first:
    unique_labels_1.add(target)

for _, target in train_dataset_filtered_second:
    unique_labels_2.add(target)

print(f"First: {unique_labels_1}")
print(f"Second: {unique_labels_2}")

First: {0, 1, 2, 3, 4, 5}
Second: {8, 9, 6, 7}


In [281]:
train_dataloader_first = DataLoader(train_dataset_filtered_first, batch_size=64, shuffle=True)
test_dataloader_first = DataLoader(test_dataset_filtered_first, batch_size=256, shuffle=False)

for X, y in train_dataloader_first:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"SHape of y: {y.shape}, dtype: {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
SHape of y: torch.Size([64]), dtype: torch.int64


In [282]:
train_dataloader_second = DataLoader(train_dataset_filtered_second, batch_size=64, shuffle=True)
test_dataloader_second = DataLoader(test_dataset_filtered_second, batch_size=256, shuffle=False)

In [283]:
eval_dataloader = DataLoader(test_dataset_3, batch_size=256, shuffle=True)

In [284]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cpu device


In [285]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes=10, hidden_size=512):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(28*28, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.classifier = nn.Linear(hidden_size, num_classes)

    #     self._initialize_weights()

    # def _initialize_weights(self):
    #     for m in self.modules():
    #         if isinstance(m, nn.Linear):
    #             nn.init.kaiming_normal_(m.weight, nonlinearity='sigmoid')
    #         elif isinstance(m, nn.Conv2d):
    #             nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        logits = self.classifier(x)

        return logits

In [286]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    pred = model(X)
    loss = loss_fn(pred, y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1) * len(X)
      print(f"Loss: {loss:>7f}, {current:>5d}/{size:>5d}")

In [287]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()

  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}, Avg Loss: {test_loss:>8f}\n")

In [288]:
pre_model = NeuralNetwork(num_classes=10, hidden_size=512).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(pre_model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [ ]:
epochs = 3
for t in range(epochs):
  print(f"Epoch {t+1}\n---------------------------")
  train(train_dataloader_first, pre_model, loss_fn, optimizer)
  test(test_dataloader_first, pre_model, loss_fn)
print("Done!")

torch.save(pre_model.state_dict(), "model_old.pth")

________

In [290]:
# model = NeuralNetwork()
# model.load_state_dict(torch.load("model_old.pth"))
model = pre_model

In [ ]:
print(list(model.named_parameters()))

In [294]:
from copy import deepcopy

def get_fisher_diag(model, dataset, params, empirical=False):
    fisher = {}
    params_dict = dict(params)
    for n, p in deepcopy(params_dict).items():
        p.data.zero_()
        fisher[n] = p.data.clone().detach().requires_grad_()

    model.eval()

    for input, gt_label in dataset:
        input, gt_label = input.to(device), gt_label.to(device)
        model.zero_grad()
        output = model(input)
        # output = output.view(1, -1)
        if empirical:
            label = output.max(1)[1]
        else:
            label = gt_label

        # label = gt_label.repeat(output.size(0))
        negloglikelihood = torch.nn.functional.nll_loss(torch.nn.functional.log_softmax(output, dim=1), label)
        negloglikelihood.backward()

        for n, p in model.named_parameters():
            fisher[n].data += p.grad.data ** 2 / len(dataset)

    fisher = {n: p for n, p in fisher.items()}
    return fisher

def get_ewc_loss(model, fisher, p_old):
    loss = 0
    for n, p in model.named_parameters():
        if n in fisher:
            _loss = fisher[n] * (p - p_old[n]) ** 2
            loss += _loss.sum()
    return loss

In [295]:
model.to(device)

ewc_lambda = 0.1

fisher_matrix = get_fisher_diag(model, train_dataloader_first, model.named_parameters())
prev_params = {n: p.data.clone() for n, p in model.named_parameters()}

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [296]:
def train(dataloader, model, loss_fn, optimizer, fisher_matrix, prev_params):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)

        # Original loss
        ce_loss = loss_fn(pred, y)

        # EWC loss
        ewc_loss = get_ewc_loss(model, fisher_matrix, prev_params)

        loss = ce_loss + ewc_lambda * ewc_loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"Loss: {loss:>7f}, {current:>5d}/{size:>5d}")

In [297]:
def val(epoch):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(eval_dataloader):
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            _, predicted_old = outputs.max(1)
            total += len(y)
            correct += predicted_old.eq(y).sum().item()
        print(f"Validation Acc: {100. * correct / total}\n")

In [ ]:
for epoch in range(3):
    print(f"Epoch {epoch+1}: ----------------------")
    train(train_dataloader_second, model, loss_fn, optimizer, fisher_matrix, prev_params)
    # Update fisher matrix and previous parameters after each epoch
    # fisher_matrix = get_fisher_diag(model, train_dataloader_second, model.named_parameters())
    # prev_params = {n: p.data.clone() for n, p in model.named_parameters()}
    test(test_dataloader_second, model, loss_fn)
    val(epoch)
print("Done!")

In [ ]:

print(fisher_matrix)